# Building AIA Chatbot by using retrieval augmented generation (RAG)

# 安裝相關套件

In [29]:
!pip install openai
!pip install python-dotenv
!pip install langchain
!pip install chromadb

In [30]:
from langchain.chains import ConversationalRetrievalChain
from langchain.chains.llm import LLMChain
from langchain.schema import Document

In [31]:
import os
import openai
import sys
sys.path.append('../..')

# 設定Emmbedings model

In [32]:
# install package
!pip install -U langchain-nomic

In [33]:
#nomic embdding 服務需要先申請 api key: https://atlas.nomic.ai
from langchain_nomic.embeddings import NomicEmbeddings
os.environ['NOMIC_API_KEY'] = 'nk-BCjzlgQXfYdZNA6kUZ-6Jeq1NIG2JhlQJaTe9BedpDc'
embedding = NomicEmbeddings(model="nomic-embed-text-v1.5")
text = "This is a test document."
query_result = embedding.embed_query(text)
query_result[:3]

[0.045318604, 0.038726807, -0.20007324]

In [34]:
#確認embedding model='nomic.embeddings
print(embedding)

In [35]:
#查看該文字轉換後的向量
# doc_result = embedding.embed_documents([text]) 完整多維度向量
query_result = embedding.embed_query(text)
query_result[:3]

[0.045318604, 0.038726807, -0.20007324]

# 建立向量資料庫

## 連線至本地端PrimeHub vectorDB

In [36]:
import chromadb
from chromadb.config import Settings
from langchain.vectorstores import Chroma
# 連線設定
httpClient = chromadb.HttpClient(
  host='64.176.47.89', port=8000,
  settings=Settings(chroma_client_auth_provider="chromadb.auth.basic_authn.BasicAuthClientProvider",chroma_client_auth_credentials="admin:admin")
  )

# collections = httpClient.list_collections() #資料庫列表
#httpClient.delete_collection(name="xt131028") #刪除資料庫
#collection = httpClient.create_collection("xt131028") #可創建個人資料庫

# tell LangChain to use our client and collection name
db = Chroma(
    client=httpClient,
    collection_name="xt131028",
    embedding_function=embedding,
)
#db4.get() #查看所有資料

In [37]:
#查詢AIA本地端db
query = "大型語言模型實作"
documents = db.similarity_search(query)
documents

[Document(page_content='大型語言模型實作進階班 (第四期) 招生簡章: 大型語言模型實作進階班 (第四期) 招生簡章🚀透過上機實作教學和專題實作導引，熟練地將這些技術應用於實際問題中，開發出具有價值的 AI 應用。💪👍🤝課程大綱立即報名課程簡介基於上百家企業需求想要用大型語言模型建置企業內部的「企業大腦」，因各企業的資料有機敏性與獨特性，無法使用公開的大型語言模型。此外微調及部署大型語言模型服務有一定的門檻在，不論是在設備上、資料處理上、技術上等。台灣人工智慧學校深知大型語言模型在當今 AI 領域的重要性，因此特別開設此課程。本課程旨在讓學員不只理解大型語言模型的基本原理，更透過上機實作教學和專題實作導引，熟練地將這些技術應用於實際問題中，開發出具有價值的 AI 應用。我們期望，透過這樣的完整培訓，能為未來的 AI 領域孕育更多的專業人才，共同推進這一革命性技術的進步。課程成果圖：學員專題成果，地端檢索增強生成 (Retrieval-Augmented Generation, RAG) 串接資料庫進行資料檢索。課程資訊修業日期2024 年 8 月 03 日', metadata={'class_name': '大型語言模型實作進階班 (第四期) 招生簡章', 'source': '簡章', 'url': 'https://aiacademy.tw/admission-llmb-tw/'}),
 Document(page_content='大型語言模型實作初階班 (第三期) 招生簡章: 和大語言模型兩種技術融合在一起，能讓您的辦公室工作更輕鬆、更高效。您可以應用大語言模型來設計和優化您的辦公室流程，讓軟體機器人能夠更準確地執行您想要的任務。您也可以使用軟體機器人來呼叫和使用大語言模型，讓您能夠更快速地完成您需要的語言相關工作。台灣人工智慧學校特別針對大型語言模型開設課程，適合無技術背景，想運用大型語言模型實現自動化流程的所有人。除了認識大型語言模型的運作原理，學習', metadata={'class_name': '大型語言模型實作初階班 (第三期) 招生簡章', 'source': '簡章', 'url': 'https://aiacademy.tw/admission-llma-tw3/'}),
 Document(page_conte

## 查詢向量資料庫的方式

In [38]:
question = "大型語言模型實作"

#以字串查詢vectordb，回傳page_content內的資料
embedding_vector=embedding.embed_query(question)
research=db.similarity_search(question,k=3)
print(research[0].page_content)
print(research[1].page_content)
print(research[2].page_content)

大型語言模型實作進階班 (第四期) 招生簡章: 大型語言模型實作進階班 (第四期) 招生簡章🚀透過上機實作教學和專題實作導引，熟練地將這些技術應用於實際問題中，開發出具有價值的 AI 應用。💪👍🤝課程大綱立即報名課程簡介基於上百家企業需求想要用大型語言模型建置企業內部的「企業大腦」，因各企業的資料有機敏性與獨特性，無法使用公開的大型語言模型。此外微調及部署大型語言模型服務有一定的門檻在，不論是在設備上、資料處理上、技術上等。台灣人工智慧學校深知大型語言模型在當今 AI 領域的重要性，因此特別開設此課程。本課程旨在讓學員不只理解大型語言模型的基本原理，更透過上機實作教學和專題實作導引，熟練地將這些技術應用於實際問題中，開發出具有價值的 AI 應用。我們期望，透過這樣的完整培訓，能為未來的 AI 領域孕育更多的專業人才，共同推進這一革命性技術的進步。課程成果圖：學員專題成果，地端檢索增強生成 (Retrieval-Augmented Generation, RAG) 串接資料庫進行資料檢索。課程資訊修業日期2024 年 8 月 03 日
大型語言模型實作初階班 (第三期) 招生簡章: 和大語言模型兩種技術融合在一起，能讓您的辦公室工作更輕鬆、更高效。您可以應用大語言模型來設計和優化您的辦公室流程，讓軟體機器人能夠更準確地執行您想要的任務。您也可以使用軟體機器人來呼叫和使用大語言模型，讓您能夠更快速地完成您需要的語言相關工作。台灣人工智慧學校特別針對大型語言模型開設課程，適合無技術背景，想運用大型語言模型實現自動化流程的所有人。除了認識大型語言模型的運作原理，學習
大型語言模型實作初階班 (第二期) 招生簡章: 和大語言模型兩種技術融合在一起，能讓您的辦公室工作更輕鬆、更高效。您可以應用大語言模型來設計和優化您的辦公室流程，讓軟體機器人能夠更準確地執行您想要的任務。您也可以使用軟體機器人來呼叫和使用大語言模型，讓您能夠更快速地完成您需要的語言相關工作。台灣人工智慧學校特別針對大型語言模型開設課程，適合無技術背景，想運用大型語言模型實現自動化流程的所有人。除了認識大型語言模型的運作原理，學習


In [39]:
#將問題變為向量，以向量去查詢vectordb，回傳page_content內的資料
embedding_vector=embedding.embed_query(question)
research = db.similarity_search_by_vector(embedding_vector,k=3)
print(research[0].page_content)
print(research[1].page_content)
print(research[2].page_content)
# len(research)

大型語言模型實作進階班 (第四期) 招生簡章: 大型語言模型實作進階班 (第四期) 招生簡章🚀透過上機實作教學和專題實作導引，熟練地將這些技術應用於實際問題中，開發出具有價值的 AI 應用。💪👍🤝課程大綱立即報名課程簡介基於上百家企業需求想要用大型語言模型建置企業內部的「企業大腦」，因各企業的資料有機敏性與獨特性，無法使用公開的大型語言模型。此外微調及部署大型語言模型服務有一定的門檻在，不論是在設備上、資料處理上、技術上等。台灣人工智慧學校深知大型語言模型在當今 AI 領域的重要性，因此特別開設此課程。本課程旨在讓學員不只理解大型語言模型的基本原理，更透過上機實作教學和專題實作導引，熟練地將這些技術應用於實際問題中，開發出具有價值的 AI 應用。我們期望，透過這樣的完整培訓，能為未來的 AI 領域孕育更多的專業人才，共同推進這一革命性技術的進步。課程成果圖：學員專題成果，地端檢索增強生成 (Retrieval-Augmented Generation, RAG) 串接資料庫進行資料檢索。課程資訊修業日期2024 年 8 月 03 日
大型語言模型實作初階班 (第三期) 招生簡章: 和大語言模型兩種技術融合在一起，能讓您的辦公室工作更輕鬆、更高效。您可以應用大語言模型來設計和優化您的辦公室流程，讓軟體機器人能夠更準確地執行您想要的任務。您也可以使用軟體機器人來呼叫和使用大語言模型，讓您能夠更快速地完成您需要的語言相關工作。台灣人工智慧學校特別針對大型語言模型開設課程，適合無技術背景，想運用大型語言模型實現自動化流程的所有人。除了認識大型語言模型的運作原理，學習
大型語言模型實作初階班 (第二期) 招生簡章: 和大語言模型兩種技術融合在一起，能讓您的辦公室工作更輕鬆、更高效。您可以應用大語言模型來設計和優化您的辦公室流程，讓軟體機器人能夠更準確地執行您想要的任務。您也可以使用軟體機器人來呼叫和使用大語言模型，讓您能夠更快速地完成您需要的語言相關工作。台灣人工智慧學校特別針對大型語言模型開設課程，適合無技術背景，想運用大型語言模型實現自動化流程的所有人。除了認識大型語言模型的運作原理，學習


In [40]:
#以向量相似度（距離）去查詢vectordb，回傳page_content內的資料
research = db.similarity_search_with_score(question,k=3)
for research in research:
  print(research[0].page_content,research[1])

大型語言模型實作進階班 (第四期) 招生簡章: 大型語言模型實作進階班 (第四期) 招生簡章🚀透過上機實作教學和專題實作導引，熟練地將這些技術應用於實際問題中，開發出具有價值的 AI 應用。💪👍🤝課程大綱立即報名課程簡介基於上百家企業需求想要用大型語言模型建置企業內部的「企業大腦」，因各企業的資料有機敏性與獨特性，無法使用公開的大型語言模型。此外微調及部署大型語言模型服務有一定的門檻在，不論是在設備上、資料處理上、技術上等。台灣人工智慧學校深知大型語言模型在當今 AI 領域的重要性，因此特別開設此課程。本課程旨在讓學員不只理解大型語言模型的基本原理，更透過上機實作教學和專題實作導引，熟練地將這些技術應用於實際問題中，開發出具有價值的 AI 應用。我們期望，透過這樣的完整培訓，能為未來的 AI 領域孕育更多的專業人才，共同推進這一革命性技術的進步。課程成果圖：學員專題成果，地端檢索增強生成 (Retrieval-Augmented Generation, RAG) 串接資料庫進行資料檢索。課程資訊修業日期2024 年 8 月 03 日 0.3446047902107239
大型語言模型實作初階班 (第三期) 招生簡章: 和大語言模型兩種技術融合在一起，能讓您的辦公室工作更輕鬆、更高效。您可以應用大語言模型來設計和優化您的辦公室流程，讓軟體機器人能夠更準確地執行您想要的任務。您也可以使用軟體機器人來呼叫和使用大語言模型，讓您能夠更快速地完成您需要的語言相關工作。台灣人工智慧學校特別針對大型語言模型開設課程，適合無技術背景，想運用大型語言模型實現自動化流程的所有人。除了認識大型語言模型的運作原理，學習 0.3475211262702942
大型語言模型實作初階班 (第二期) 招生簡章: 和大語言模型兩種技術融合在一起，能讓您的辦公室工作更輕鬆、更高效。您可以應用大語言模型來設計和優化您的辦公室流程，讓軟體機器人能夠更準確地執行您想要的任務。您也可以使用軟體機器人來呼叫和使用大語言模型，讓您能夠更快速地完成您需要的語言相關工作。台灣人工智慧學校特別針對大型語言模型開設課程，適合無技術背景，想運用大型語言模型實現自動化流程的所有人。除了認識大型語言模型的運作原理，學習 0.34922096133232117


# 建立 PromptTemplate

In [41]:
# 建立 PromptTemplate
from langchain.prompts import PromptTemplate

# template = """You are a customer support agent. You are designed to be as helpful as possible while providing only factual information. You should be friendly, but not overly chatty. Context information is below. Given the context information and not prior knowledge, answer the query. Always say "thanks for asking!" at the end of the answer.
# template = """Context information is below. Given the context information and not prior knowledge. Please answer the question by zh-tw language If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer.
template = """Use the following pieces of context to answer the question at the end. Please answer in Chinese. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer.
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"],template=template)


# 建立要使用的 LLM Model


## Call Groq

In [42]:
!pip install langchain-groq
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

In [43]:

chat = ChatGroq(temperature=0, groq_api_key="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", model_name="llama3-70b-8192")
#MODEL_NAME = "llama3-70b-8192" or "llama3-8b-8192"

system = "You are a helpful assistant."
human = "{text}"
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

chain = prompt | chat
chain.invoke({"text": "什麼是大型語言模型？"})

AIMessage(content="\n\nA large language model (LLM) is a type of artificial intelligence (AI) model that is trained on a massive amount of text data to generate language understanding and generation capabilities. These models are typically based on deep learning architectures, such as transformers, and are designed to process and analyze vast amounts of text data.\n\nThe key characteristics of a large language model are:\n\n1. **Scale**: LLMs are trained on enormous datasets, often consisting of billions of parameters and millions of examples.\n2. **Language understanding**: LLMs are designed to understand natural language, including syntax, semantics, and pragmatics.\n3. **Generation capabilities**: LLMs can generate human-like text, including text completion, language translation, and text summarization.\n\nSome examples of large language models include:\n\n1. BERT (Bidirectional Encoder Representations from Transformers)\n2. RoBERTa (Robustly Optimized BERT Pretraining Approach)\n3.

# 建立問答鏈 RetrievalQA

In [47]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

# question = "大型語言模型實作進階班的開課日期是什麼時候？"
# question = "摘要技術領袖培訓全域班課程"
question = "介紹大型語言模型實作進階班"

qa_chain = RetrievalQA.from_chain_type(llm=chat,
                                       retriever=db.as_retriever(),
                                       return_source_documents=True,
                                       chain_type="stuff",
                                       chain_type_kwargs={"prompt": QA_CHAIN_PROMPT})

result = qa_chain({"query": question})
result["result"] #查看結果

'大型語言模型實作進階班旨在讓學員不只理解大型語言模型的基本原理，更透過上機實作教學和專題實作導引，熟練地將這些技術應用於實際問題中，開發出具有價值的 AI 應用。thanks for asking!'

# AIA FAQ資料

In [ ]:
# 註冊繳費
question: 我已完成報名繳費,何時可以收到發票?
answer: 個人用電子發票會於開學後第二週上課日由學務通知領取。公司報帳用電子發票會於開學後第二週由學務 mail 至學員報名時的電子信箱。若需提早或延後開立發票,請發信至hi@aiacademy.tw詢問。

question: 我已過繳費期限?
answer: 因原先繳費期限已過期,請發信至 hi@aiacademy.tw 並提供您的姓名及報名時填寫的電子郵件信箱。我們將會重新寄發{報名及註冊的登記已可進行下一步報名作業}信件至您的電子信箱。

question: 註冊後,我選擇臨櫃匯款,該匯到哪裡?
answer: 選擇臨櫃匯款者,請匯款至 : 玉山銀行 南港分行 金融機構代碼：808 1182 戶名：財團法人台灣人工智慧學校基金會 帳號：1182-940-016585。注意 : 選擇臨櫃匯款繳費的學員請務必於匯款完成後來信至 hi@aiacademy.tw 提供您的姓名、報名課別、匯款單或匯款帳號後五碼,以利我們對帳後更新你的註冊狀態。

question: 如何繳費?
answer: 您好,請發信至 hi@aiacademy.tw 並提供您的姓名及報名時填寫的電子郵件信箱。我們會於系統更新你的審核狀態並重新寄發{報名及註冊的登記已可進行下一步報名作業}至您的電子信箱。若您報名的是經理人週末研修班或技術領袖全域班獲錄取者,請於收到錄取通知後 3 天內完成註冊繳費。繳費方式可選擇線上金流 (信用卡) 或臨櫃匯款。若選擇臨櫃匯款者,請匯款至 : 玉山銀行 南港分行 金融機構代碼：808 1182 戶名：財團法人台灣人工智慧學校基金會 帳號：1182-940-016585。\n注意 : 選擇臨櫃匯款繳費的學員請務必於匯款完成後來信至 hi@aiacademy.tw 提供匯款單或匯款帳號後五碼,以利我們對帳後更新你的註冊狀態。

# 退費
question: 我要如何申請退費?
answer: 很遺憾您無法與我們一起為接下來的課程努力。\請您發信至hi@aiacademy.tw 提供您的班別跟姓名。 退費的標準,我們會依照學員手冊退費辦法辦理,後續也請提供你您帳號(含分行)資訊(存摺影本),以利我們後續匯款。謝謝

question: 申請退費
answer: 親愛的學員你好,很遺憾你無法與我們一起為接下來的課程努力。退費的標準,我們會依照學員手冊退費辦法辦理。此外,請將學員證與發票退回至行政人員,也請告知匯款帳戶,以利我們作業。

# 請假
question: 如何請假?
answer: 若學員因故有請假需求,請填妥請假表單進行申請 https://reurl.cc/xZNOAL校務行政處核可後,系統將會直接寄送請假核可通知至您的信箱中。請假申請審核過後,仍計入缺席時數。本課程無補課機制。

#報名
question: 我的報名已過期?
answer: 因原先報名連結已過期,請發信至 hi@aiacademy.tw 並提供您的姓名及報名時填寫的電子郵件信箱。我們將會重新寄發{報名及註冊的登記已可進行下一步報名作業}信件至您的電子信箱。

question: 我是文科的人,我可以報名你們的課程嗎？
answer: 我們非常歡迎不同產業的人進來,希望可以推動不同產業專業的人,能認識AI這些知識。

#學費補助
question: 請問你們有學費補助嗎？因為我看我們醫院有補助計畫,那如果你們這一期沒有,我是否可以轉下一期？
answer: 我們學校沒有提供學費補助,應該是貴醫院單位的補助計畫,可能需要您與貴單位做內部確認,是否能申請院內補助,我本校無權涉入。

#電腦
question: 課程是否需要帶電腦?
answer: 課程當中並沒有需要使用到電腦的部(除了工程技術類課程外),您可以依據自身上課需要攜帶筆電。

#午餐
question: 請問你們所有課程都有提供午餐嗎？
answer: 因為我們的班別有經理人周末研修班、技術領袖全域班、智慧醫療專班和產業專班,目前只有經理人周末研修班/智慧醫療專班和產業專班有提供午餐。

#結業
question: 學員要通過那些門檻才會有結業證書？是否有課綱介紹？
answer: 結業門檻等相關行政細節資訊列於學員手冊上,待開學當周會寄發給各位學員。課綱可以先參考官網 https://aiacademy.tw/

#課程差別
question: 醫療專班跟技術班還有經理人班的差別是？
answer: 醫療專班：智慧醫療專班是台灣人工智慧學校為台灣醫療產業從業人員所設計的密集課程,期待透過智慧醫療專班,讓醫療產業從業人員能夠快速、系統化掌握人工智慧技術的最新趨勢,並徹底瞭解人工智慧如何運作,以及它的能力、侷限及未來發展,並能實際應用在醫療工作的每個場景。\n智慧金融專班:\n給予專班學員金融業全面性應用到的人工智慧知識,再透過實務議題、案例分享與場域實地參訪的印證,即時掌握智慧金融應用的趨勢。同時安排學員們進行深入的專題討論,將課堂傳授的知識運用於專題及實作,領略這些技術的實際應用方式及限制,以及進行學員間深入的交流學習。\n經理人班：\n經理人週末研修班是台灣人工智慧學校在技術領袖培訓班之後,第二個專門為台灣目前產業需求所設計的班別。我們預期透過經理人週末研修班,讓各產業的經理人能有一個場域,與不同產業的經理人一同學習及理解人工智慧的技術概觀,對於人工智慧技術能有全面性的大局觀,而且徹底瞭解人工智慧是如何運作的,以及它的能力、侷限及未來發展,才能讓產業欲以人工智慧進行產業升級時,能有清楚的方向感來帶領企業前進。\n技術班：\n技術領袖培訓班期為台灣育成具有人工智慧技術思維與實戰經驗的技術領袖人才,讓產業欲以人工智慧進行產業升級時,不再為缺乏人才所束縛。

#師資
question: 詢問智慧醫療專班師資
answer: 台灣人工智慧學校提供專業的師資與課程,也安排醫界的先進前來授課,分享智慧醫療應用與案例的分享。希望能透過五週的課程,讓工作繁忙的醫師及護理相關人員,能快速、系統化掌握人工智慧技術的最新趨勢與發展應用能力,同時進行醫療實作案例分享。\n學校將邀請中央研究院等學研單位及產業界,擁有豐富經驗與理論背景的講師,進行機器學習、深度學習、智慧生醫等重要的人工智慧核心技術與應用課程講授。\n預計將培養上千位醫療產業從業人員,擁有定義問題、解決及場域導入方式等關鍵能力,並能實際應用在醫療工作上為全台醫療產業注入維持競爭力的能量。\n本次北部醫療專班的師資\n陳弘軒        國立中央大學 資工系副教授        機器學習\n蔡炎龍        國立政治大學應用數學系副教授        深度學習\n莊永裕        台大資工系 教授        電腦視覺\n蔣榮先         成大資工系教授暨醫院健康數據中心執行長        智慧醫療應用\n及其他醫界先進們,目前講師還在安排調整,官網只會放上課程大綱,待開課前會寄出學員手冊,裡面的課程表會有當期課程及講師,屆時再提供您參考\n

question: 詢問智慧製造專班師資
answer: 台灣人工智慧學校提供專業的師資與課程,也安排了在製造業有深厚實務經驗的先進前來授課,分享智慧製造應用與案例的分享。希望能透過二天共七週的課程,讓來參與智慧製造課程的各位,能充份吸收並掌握人工智慧技術的最新趨勢與發展應用能力。\n學校將邀請中央研究院等學研單位及產業界,擁有豐富經驗與理論背景的講師,進行機器學習、深度學習、智慧製造等重要的人工智慧核心技術與應用課程講授。\n本次產業專班的師資\n吳漢銘        國立政治大學統計學系副教授      資料科學與大數據分析\n陳弘軒       國立中央大學 資工系副教授        機器學習與演算法概論\n許懷中       逢甲大學資訊工程學系副教授     深度學習 \n莊永裕       台大資工系 教授                          電腦視覺 \n李家岩       國立臺灣大學資訊管理學系教授  智慧製造與生產線上的資料科學\n智慧製造實務案例分享及應用,目前講師還在安排調整,官網只會放上課程大綱,待開課前會寄出學員手冊,裡面的課程表會有當期課程講義及授課講師,屆時再提供您參考

question: 詢問經理人班師資
answer: 台灣人工智慧學校提供專業的師資與課程,也安排有深厚實務經驗的先進前來授課,分享人工智慧的應用與案例的分享。希望能透過14週的課程,從基礎核心、實作學習到產業應用,讓來參與課程的學員,充份吸收並掌握人工智慧技術的最新趨勢與發展應用能力。   學校將邀請學研單位及產業界,擁有豐富經驗與理論背景的講師,進行機器學習、深度學習、電腦視覺、資料探勘等重要的人工智慧核心技術與應用課程講授。 本次經理人班的師資 吳漢銘 國立政治大學統計學系副教授 資料科學與大數據分析 陳弘軒 國立中央大學 資工系副教授 機器學習與演算法概論 蔡炎龍 國立政治大學應用數學系副教授 深度學習 莊永裕 國立臺灣大學資訊工程學教授 電腦視覺 陳宜欣 國立清華大學資訊工程學系教授 資料探勘 （學校保有修改、變更課程內容之權利）\n以及業界先進產業案例的分享及應用。\n官網只會放上課程大綱,待開課前會寄出學員手冊,登入學員專區裡面的課程表,會有當期課程講義及授課講師。\n本次經理人班也加入AIGC實戰應用,學會使用相關工具如：ChatGPT、 Midjourney 等來生成文字與圖像,結合情境應用,有效減輕工作負擔,瞬間提升工作效率。

#發票
question: 學員發票什麼時候拿
answer: 個人用電子發票會於開學後第二週上課日由學務通知領取。\r公司報帳用電子發票會於開學後第二週由學務 mail 至學員報名時的電子信箱。\r若需提早或延後開立發票,請發信至hi@aiacademy.tw詢問。發票部分,若有疑慮,請於五日內向校方反應。\n電子發票會於今晚上傳財政部申報後,寄發至你的電子信箱中。

#開學典禮
question: 明天開學典禮我無法參加,需要請假嗎？
answer: 開學典禮及結業典禮皆為課程時間,出缺席將計入缺席時數。若學員因故有請假需求,請填妥請假表單進行申請：https://reurl.cc/xZNOAL,並上傳證明文件,行政處核可後,系統將會直接寄送請假核可通知至您的信箱中。提醒您於下次上課時,攜帶有照片的證件,來領取學員證。

question: 結業相關問題 我當天結業無法出席怎麼辦？
answer: 開學典禮及結業典禮皆為課程時間,出缺席將計入缺席時數。若學員因故有請假需求,請填妥請假表單進行申請：https://reurl.cc/xZNOAL,並上傳證明文件,行政處核可後,系統將會直接寄送請假核可通知至您的信箱中。

#退學
question: 如果我退學了,我還可以登入專區進去看影片等教學相關素材嗎？
answer: 這邊和您說聲抱歉。退學或是終止學員,將無法進入專區觀看相關影片,以維護其他學員的就學權益。

#轉班
question: 開學前申請轉班
answer: 技術班轉經理人班,我們需要檢視您的職級職位作審核。經理人班轉技術班,我們會看您的背景及程式能立,方能決定是否安排入學考試。

question: 開學後申請轉班
answer: 1.除非特殊情況,依學員手冊規定,不許轉班\n2.特殊情況的話(像是個人不可抗拒之因素),需要學員提出證明,學校方得受理 3.課程進行兩周之後,也不許轉班

#補刷
question: 如何補刷
answer: 可使用報到處的平板補刷或是發信至hi@aiacademy.tw /私訊台灣人工智慧學校學務,並提供你的學號/姓名及補刷時間,即可。以上資訊供您參考,若您還有其他疑問也歡迎再與我們聯繫。

#競賽
question: 請問要結業的其中一項是繳交競賽的報告,若團隊中有人幾乎0互動,在現在幾乎是線上上課也只能線上互動的,好像完全沒有約束力
answer: 請組長分配給所有組員該負責的任務,若有組員拒不合作,再請組長私訊告知,我們再去與該學員私下勸說。我們又會有組員互評機制。謝謝

question: 請問每人需參與 AI 產業創新競賽？
answer: 參加AI創新競賽是結業的其中一個門檻。

#颱風
question: 如果遇到颱風天,你們課程會順延嗎？
answer: 如遇颱風天,本校活動、上課是否照常進行,將依行政院人事行政局公布之臺北市停班停課標準為主,相關消息將另行公告於官網、官方臉書。

#學員證
question: 我的學員證不見了,要申請,要怎麼給你們錢?
answer: 您好,請發信至hi@aiacademy.tw 申請,以便我們作業紀錄登記；若您確定申請補發學員證,我們將製作好之後,會現場給您學員證+發票,屆時會與您收500元。

#校友身分
question: 如何核實校友身分?
answer: 具備台灣人工智慧學校結業證書者。

#學號
question: 要如何查詢我的學號?
answer: 請至官網,點選校友資源內的學號查詢即可。

#結業證書
question: 數位結業證書申請補發?
answer: 申請證書,需收取500元申請費用,將開立發票,請提供發票寄件地址。

question: 結業證書寄送問題
answer: 學校將於結業典禮後㇐週內以電子郵件寄發圖靈數位證書至您提供的電子信箱

question: 未收到數位結業證書?
answer: 請至證書入口網站https://global.turingcerts.com/login (使用者登入)使用自己受訓登記的電子郵件註冊進入,即可看到自己證書。第一次登入需要先註冊,電子郵件以報名時的E-mail信箱。註冊後的登入頁面,雖然要求以手機號碼登入,但實質上是使用受訓登記的電子郵件進行登入~  可詳閱收證方使用手冊 https://taiwan-e-portfolio-alliance.gitbook.io/turing-certs-user-guidebook/


#技術領袖全域班 入學考試
question: 何時會寄發技術領袖全域班入學考試連結?
answer: 技術領袖全域班考試連結會於考試前一天17:00前寄發連結並加發簡訊給應考者,線上入學考的連結於考試開始方可登入。

question: 為何我未通過技術領袖全域班入學考試?
answer: 謝謝您的來信。很遺憾您未通過技術領袖全域班入學考試。試題中,選擇題與程式題各佔50分,總分為100分,本期考生的程度都相當好,在參酌其他考生的成績後,我們會擇優錄取。

question: 想報考技術領袖全域班,請問一下 有考古題嗎?
answer: 我們沒有考古題。入學考目前focus在基礎的Python, 所以不管是坊間的參考書,或是學校官方網站的建議內容,應該都可以讓學員有能力通過入學考。謝謝

question: 請問您們技術領袖全域班上課幾周? 總時數為?
answer: 技術領袖全域班上課約十五周,為每周星期三、星期五和星期六,時間為早上 09:30到下午17:00,總計上課時數約 270小時。技術領袖全域班學員缺席達十分之一上課時數者,不予發給結業證書；您也可以參閱官網 https://aiacademy.tw/的招生訊息內的資訊。


#產業專班
question: 為何我未錄取產業專班?
answer: 謝謝您的來信。由於報名實在太熱烈及踴躍、報考者都十分優秀,我們能錄取的考生非常有限,不免會有遺珠之憾,對於產業專班,校方希望提供給在職人士進修與推動產業轉型AI機會, 台灣人工智慧學校經理人班審查標準有二： 1. 從事AI相關工作經理人 2. 具備決策權與預算權的經理人 3. 若以上兩者符合,根據職稱高低以及公司產業別排序 非常的可惜未能錄取所有優秀的報考者,我們期待您未來的參與。建議報考者能就近多利用其他分校,希望及歡迎您報名其他分校或再次報考下一梯次的產業專班。

#專題實作班
question: 請問您們專題實作班上課幾周? 總時數為?
answer: 專題實作班上課五周,為每周星期四和星期五晚間19:00到21:00,星期六時間為早上 09:00到下午17:00,總計上課時數約 46.5小時。您也可以參閱官網 https://aiacademy.tw/的招生訊息內的資訊。

question: 關於【 NLP 專題實作班 】入學考試型式與範圍?
answer: 關於入學考試型式與範圍概述如下：1. 選擇題 10-15 題 ： 機器學習 / 深度學習 / NLP / Python / PyTorch 之基礎概念 2. 手寫題 3 題 ： Python / PyTorch 不要求全對 /  有寫的要正確即可\n以上請參考並保持平常心即可！

question: 請問您們專題實作班上課幾周? 總時數為?
answer: 產業專班上課七周,為每周星期五和星期六,時間為早上 09:00-到下午17:30,總計上課時數為 105小時。您也可以參考官網 https://aiacademy.tw/的招生訊息內的資訊。

#智慧醫療專班結業門檻
question: 結業門檻為何？
answer: 智慧醫療專班學員需參與醫療專班創新競賽且缺席時數不超過15小時(含)即可取得結業證書。產業專班及經理人周末研修班學員需參與 AI 產業創新競賽且缺席時數不超過27小時(含)。

question: 請問您們智慧醫療專班上課幾周? 總時數為?
answer: 智慧醫療專班上課五周,為每星期六,時間為早上 09:30到下午18:00,總計上課時數為 37.5小時。您也可以參考官網 https://aiacademy.tw/的招生訊息內的資訊。

question: 請問智慧醫療專班這門課有要求學員會哪一種程式語言嗎?謝謝
answer: 智慧醫療專班這門課是為了要讓醫學相關領域的專業人士能更快速的掌握人工智慧的知識及資訊,而非專精於程式語言的學習。因此並無特別需要學會哪一種程式語言。相關的課程訊息也歡迎您參考：https://aiacademy.tw/curriculum-med/\n若還有其他的疑問也歡迎與我們聯繫。


#經理人週末研修班
question: 為何我未錄取經理人週末研修班?
answer: 謝謝您的來信。由於報名實在太熱烈及踴躍、報考者都十分優秀,我們能錄取的考生非常有限,不免會有遺珠之憾,對於經理人週末研修班,校方希望提供給在職人士進修與推動產業轉型AI機會, 台灣人工智慧學校經理人班審查標準有二： 1. 從事AI相關工作經理人 2. 具備決策權與預算權的經理人 3. 若以上兩者符合,根據職稱高低以及公司產業別排序 非常的可惜未能錄取所有優秀的報考者,我們期待您未來的參與。建議報考者能就近多利用其他分校,希望及歡迎您報名其他分校或再次報考下一梯次的經理人週末研修班。

question: 經理人週末研修班需要考試或審核嗎？怎麼審核？審核標準是什麼?
answer: 參加經理人週末研修班是不需要經過考試。但需要經過審核,審核的標準為 1. 從事AI相關工作經理人2. 具備決策權與預算權的經理人3. 若以上兩者符合,根據職稱高低排

question: 請問您們經理人週末研修班上課幾週? 總時數為?
answer: 經理人週末研修班上課十四週,為每週星期六,時間為早上 09:00到下午17:30,總計上課時數約 105小時。經理人週末研修班學員缺席達四分之一上課時數者,不予發給結業證書；您也可以參閱官網 https://aiacademy.tw/的招生訊息內的資訊。

